# DSC Donation Details

### Import Libraries

In [755]:
import sys
import pandas
import numpy
from pandas               import DataFrame
from pandas               import read_csv
from numpy                import where
from numpy                import array
from numpy                import random
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics      import auc
from sklearn.metrics      import roc_auc_score
from matplotlib           import pyplot
from scipy.stats          import pearsonr

### Local path

In [756]:
sys.path.append("C:/Users/sbadhani/Desktop/Predictive Analysis")

In [757]:
from ieseg import partition
from ieseg import roc

### Path

In [758]:
directory = "C:/Users/sbadhani/Desktop/Predictive Analysis/Project_Data"
inputData = directory + "/Data"
sandbox   = directory + "/Sandbox"

### Import Data

In [759]:
# Train dataset
campaign2013 = read_csv(inputData + "/campaign20130411.csv", sep = ';')

In [760]:
#Grouping the data by DonorID to find the total amount donated
campaign2013 = campaign2013.groupby(['donorID'],as_index=False).agg('sum')

In [761]:
#Rename Amount Column
#Column 1: amount_campaign
campaign2013 = campaign2013.rename(columns= {'amount': 'amount_campaign'})

In [762]:
# Donor dataset
donors = read_csv(inputData + "/donors.csv", sep = ';', na_values = 'Missing') #Assigning missing values

In [763]:
#columns Added for Gender
#Coumn 2,3,4,5,6
donors['is_donor_M'] = where(donors["gender"] =='M', 1,0)
donors['is_donor_F'] = where(donors["gender"] =='F', 1,0)
donors['is_donor_C'] = where(donors["gender"] =='C', 1,0)
donors['is_donor_S'] = where(donors["gender"] =='S', 1,0)
donors['is_donor_U'] = where(donors["gender"] =='U', 1,0)

In [764]:
#column Added for Language
#Coumn 7
donors['is_donor_French'] = where(donors["language"] =='F', 1,0)


In [765]:
#Handle Null Colums in Zipcode
donors['zipcode'] = where(donors["zipcode"].isnull(), 0,donors["zipcode"])

In [766]:
#Handle one string value in Zipcode(No such zipcode in Brussel)
donors['zipcode'] = where(donors["zipcode"] =='SW6 3PS',"10000",donors['zipcode'])

In [767]:
#Change Zipcode type to Numeric
donors['zipcode']= pandas.to_numeric(donors['zipcode'])

In [768]:
donors['province'] = "London"

In [769]:
#Assign real values to province using Zipcode
donors['province'] = where((donors['zipcode'] >= 1000) & (donors['zipcode']<=1299), "Brussels capital Region",donors['province'])
donors['province'] = where((donors['zipcode'] >= 1300) & (donors['zipcode']<=1499), "Walloon Brabant",donors['province'])
donors['province'] = where((donors['zipcode'] >= 1500) & (donors['zipcode']<=1999), "Flemish Brabant",donors['province'])
donors['province'] = where((donors['zipcode'] >= 2000) & (donors['zipcode']<=2999), "Antwerp",donors['province'])
donors['province'] = where((donors['zipcode'] >= 3000) & (donors['zipcode']<=3499), "Flemish Brabant2",donors['province'])
donors['province'] = where((donors['zipcode'] >= 3500) & (donors['zipcode']<=3999), "Limburg",donors['province'])
donors['province'] = where((donors['zipcode'] >= 4000) & (donors['zipcode']<=4999), "Liège",donors['province'])
donors['province'] = where((donors['zipcode'] >= 5000) & (donors['zipcode']<=5999), "Namur",donors['province'])
donors['province'] = where((donors['zipcode'] >= 6000) & (donors['zipcode']<=6599), "Hainaut",donors['province'])
donors['province'] = where((donors['zipcode'] >= 6600) & (donors['zipcode']<=6999), "Luxembourg",donors['province'])
donors['province'] = where((donors['zipcode'] >= 7000) & (donors['zipcode']<=7999), "Hainaut2",donors['province'])
donors['province'] = where((donors['zipcode'] >= 8000) & (donors['zipcode']<=8999), "West Flanders",donors['province'])
donors['province'] = where((donors['zipcode'] >= 9000) & (donors['zipcode']<=9999), "East Flanders",donors['province'])

In [770]:
#Column Conversion for Province
donors['Is_Brussel'] = where(donors['province'] == "Brussels capital Region", 1,0)
donors['Is_Walloon'] = where(donors['province'] == "Walloon Brabant", 1,0)
donors['Is_FlemishBrabant'] = where(donors['province'] == "Flemish Brabant", 1,0)
donors['Is_Antwerp'] = where(donors['province'] == "Antwerp", 1,0)
donors['Is_FlemishBrabant2'] = where(donors['province'] == "Flemish Brabant2", 1,0)
donors['Is_Limburg'] = where(donors['province'] == "Limburg", 1,0)
donors['Is_Liège'] = where(donors['province'] == "Liège", 1,0)
donors['Is_Namur'] = where(donors['province'] == "Namur", 1,0)
donors['Is_Hainaut'] = where(donors['province'] == "Hainaut", 1,0)
donors['Is_Luxembourg'] = where(donors['province'] == "Luxembourg", 1,0)
donors['Is_Hainaut2'] = where(donors['province'] == "Hainaut2", 1,0)
donors['Is_WestFlanders'] = where(donors['province'] == "West Flanders", 1,0)
donors['Is_EastFlanders'] = where(donors['province'] == "East Flanders", 1,0)


In [771]:
#Assign real values to region based on province
donors['region'] = where(donors['province'] == "Brussels capital Region", "Brussels",donors['region'])
donors['region'] = where(donors['province'] == "Walloon Brabant", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Liège", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Namur", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Luxembourg", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Hainaut", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Hainaut2", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Flemish Brabant", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "Antwerp", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "Flemish Brabant2", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "Limburg", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "West Flanders", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "East Flanders", "Flanders",donors['region'])

In [772]:
#Column Conversion for Region
donors['Is_Brussel_region'] = where(donors['region'] == "Brussels", 1,0)
donors['Is_Wallonia_region'] = where(donors['region'] == "Wallonia", 1,0)
donors['Is_Flanders_region'] = where(donors['region'] == "Flanders", 1,0)

In [773]:
# gifts dataset
gifts = read_csv(inputData + "/gifts.csv", sep = ';')

In [774]:
#renaming the amount column in Gifts
gifts = gifts.rename(columns= {'amount': 'amount_gifts'})

In [775]:
#Convert to date format
gifts['date'] = pandas.to_datetime(gifts['date'])

In [776]:
#filtering data before the campaign date
gifts = gifts[gifts.date <= '2013-04-11']

In [777]:
camp_count =gifts.groupby(['donorID'], as_index=False)['campID'].count()
camp_count=camp_count.rename(columns= {'campID': 'Total_camps_attended'})

In [778]:
#column 8
camp_count['is_camp_attended'] = where(camp_count["Total_camps_attended"] > 0, 1,0)#not required

In [779]:
#column 9
#Total Donation per donor
donation_donor =gifts.groupby(['donorID'], as_index=False)['amount_gifts'].sum()
donation_donor=donation_donor.rename(columns= {'amount_gifts': 'Total_Amount_Donated'})

In [780]:
#create a new column for year to obtain the donations per year
gifts['gifts_year'] = gifts['date'].dt.year

In [781]:
#filtering data for the 3 year frequency rate
freq_3 = gifts[(gifts['date'] >= '2010-04-11') & (gifts['date'] <= '2013-04-11')]
#column 11
#Camp count freq per 3 years
freq_3_camp = freq_3.groupby(['donorID'], as_index=False)['campID'].count()
freq_3_camp['is_freq_3_camp_attended'] = where(freq_3_camp["campID"] >0, 1,0)
#column 12
#Donation freq per 3 years
Donation_last_3_years = freq_3.groupby(['donorID'],as_index=False)['amount_gifts'].sum()
Donation_last_3_years=Donation_last_3_years.rename(columns= {'amount_gifts': 'Donation_last_3_years'})


In [782]:
#filtering data for the 5 year frequency rate
freq_5 = gifts[(gifts['date'] >= '2008-04-11') & (gifts['date'] <= '2013-04-11')]
#column 13
##Camp count freq per 5 years
freq_5_camp = freq_5.groupby(['donorID'],as_index=False)['campID'].count()
freq_5_camp['is_freq_5_camp_attended'] = where(freq_5_camp["campID"] >0, 1,0)
#column 14
#Donation freq per 5 years
Donation_last_5_years = freq_5.groupby(['donorID'],as_index=False)['amount_gifts'].sum()
Donation_last_5_years=Donation_last_5_years.rename(columns= {'amount_gifts': 'Donation_last_5_years'})

In [783]:
#Drop Unwanted Columns
freq_5_camp=freq_5_camp.drop(['campID'], axis=1)
freq_3_camp=freq_3_camp.drop(['campID'], axis=1)

In [784]:
#column 15
#max donation by a donor yearwise
max_donation_year = gifts.groupby(['donorID','gifts_year'],as_index=False).sum()
max_donation_year = max_donation_year.groupby(['donorID'],as_index=False)['amount_gifts'].max()
max_donation_year=max_donation_year.rename(columns= {'amount_gifts': 'max_donation_year'})

In [785]:
#column 16
#min donation by a donor
min_donation_year = gifts.groupby(['donorID','gifts_year'],as_index=False).sum()
min_donation_year = min_donation_year.groupby(['donorID'],as_index=False)['amount_gifts'].min()
min_donation_year=min_donation_year.rename(columns= {'amount_gifts': 'min_donation_year'})

In [786]:
#column 18
#Max Amount Paid by a donor
Max_Amount_Donated=gifts.groupby(['donorID'],as_index=False)['amount_gifts'].max()
Max_Amount_Donated=Max_Amount_Donated.rename(columns= {'amount_gifts': 'Max_Amount_Donated'})
#column 19
#Min Amount Paid by a donor
Min_Amount_Donated=gifts.groupby(['donorID'],as_index=False)['amount_gifts'].max()
Min_Amount_Donated=Min_Amount_Donated.rename(columns= {'amount_gifts': 'Min_Amount_Donated'})


In [787]:
#column 21
#Fist Amount Donated
First_Amount_Donated=gifts.groupby(['donorID'],as_index=False)[['date']].min()
First_Amount_Donated=pandas.merge(First_Amount_Donated, gifts[['donorID', 'date', 'amount_gifts']], left_on=['donorID','date'], right_on = ['donorID','date'])
First_Amount_Donated=First_Amount_Donated.rename(columns= {'amount_gifts': 'First_Amount_Donated'})
First_Amount_Donated=First_Amount_Donated.groupby(['donorID','date'],as_index=False)['First_Amount_Donated'].sum()
#column 22
#Last Amount Donated
Last_Amount_Donated=gifts.groupby(['donorID'],as_index=False)[['date']].max()
Last_Amount_Donated=pandas.merge(Last_Amount_Donated, gifts[['donorID', 'date', 'amount_gifts']], left_on=['donorID','date'], right_on = ['donorID','date'])
Last_Amount_Donated=Last_Amount_Donated.rename(columns= {'amount_gifts': 'Last_Amount_Donated'})
Last_Amount_Donated=Last_Amount_Donated.groupby(['donorID','date'],as_index=False)['Last_Amount_Donated'].sum()

In [788]:
#column 23
#Time since first donation
Years_from_first_donation = First_Amount_Donated
Years_from_first_donation['Year']=Years_from_first_donation['date'].dt.year
Years_from_first_donation['Year']=2013-Years_from_first_donation['Year']
Years_from_first_donation=Years_from_first_donation.rename(columns= {'Year': 'Years_from_first_donation'})

In [789]:
#column 24
#Time since last donation
Years_from_last_donation = Last_Amount_Donated
Years_from_last_donation['Year']=Years_from_last_donation['date'].dt.year
Years_from_last_donation['Year']=2013-Years_from_last_donation['Year']
Years_from_last_donation=Years_from_last_donation.rename(columns= {'Year': 'Years_from_last_donation'})

In [790]:
#Drop Unwanted Columns
Years_from_last_donation=Years_from_last_donation.drop(['date'], axis=1)
Years_from_first_donation=Years_from_first_donation.drop(['date'], axis=1)
Years_from_first_donation=Years_from_first_donation.drop(['First_Amount_Donated'], axis=1)
Years_from_last_donation=Years_from_last_donation.drop(['Last_Amount_Donated'], axis=1)

In [791]:
train = campaign2013

In [792]:
training=pandas.merge(train, donors, on='donorID', how='left')
training=pandas.merge(training, camp_count, on='donorID', how='left')
training=pandas.merge(training, donation_donor, on='donorID', how='left')
training=pandas.merge(training, freq_3_camp, on='donorID', how='left')
training=pandas.merge(training, Donation_last_3_years, on='donorID', how='left')
training=pandas.merge(training, freq_5_camp, on='donorID', how='left')
training=pandas.merge(training, Donation_last_5_years, on='donorID', how='left')
training=pandas.merge(training, max_donation_year, on='donorID', how='left')
training=pandas.merge(training, min_donation_year, on='donorID', how='left')
training=pandas.merge(training, Max_Amount_Donated, on='donorID', how='left')
training=pandas.merge(training, Min_Amount_Donated, on='donorID', how='left')
training=pandas.merge(training, First_Amount_Donated, on='donorID', how='left')
training=pandas.merge(training, Last_Amount_Donated, on='donorID', how='left')
training=pandas.merge(training, Years_from_first_donation, on='donorID', how='left')
training=pandas.merge(training, Years_from_last_donation, on='donorID', how='left')


In [793]:
#Drop Unwanted Columns
training=training.drop(['gender','language','zipcode','region','province','date_x','Year_x','date_y','Year_y'], axis=1)

In [794]:
#Handle Null Colums in Zipcode
training['Total_camps_attended'] = where(training["Total_camps_attended"].isnull(), 0,training["Total_camps_attended"])
training['is_camp_attended'] = where(training["is_camp_attended"].isnull(), 0,training["is_camp_attended"])
training['Total_Amount_Donated'] = where(training["Total_Amount_Donated"].isnull(), 0,training["Total_Amount_Donated"])
training['is_freq_3_camp_attended'] = where(training["is_freq_3_camp_attended"].isnull(), 0,training["is_freq_3_camp_attended"])
training['Donation_last_3_years'] = where(training["Donation_last_3_years"].isnull(), 0,training["Donation_last_3_years"])
training['is_freq_5_camp_attended'] = where(training["is_freq_5_camp_attended"].isnull(), 0,training["is_freq_5_camp_attended"])
training['Donation_last_5_years'] = where(training["Donation_last_5_years"].isnull(), 0,training["Donation_last_5_years"])
training['max_donation_year'] = where(training["max_donation_year"].isnull(), 0,training["max_donation_year"])
training['min_donation_year'] = where(training["min_donation_year"].isnull(), 0,training["min_donation_year"])
training['Max_Amount_Donated'] = where(training["Max_Amount_Donated"].isnull(), 0,training["Max_Amount_Donated"])
training['Min_Amount_Donated'] = where(training["Min_Amount_Donated"].isnull(), 0,training["Min_Amount_Donated"])
training['First_Amount_Donated'] = where(training["First_Amount_Donated"].isnull(), 0,training["First_Amount_Donated"])
training['Last_Amount_Donated'] = where(training["Last_Amount_Donated"].isnull(), 0,training["Last_Amount_Donated"])
training['Years_from_first_donation'] = where(training["Years_from_first_donation"].isnull(), 0,training["Years_from_first_donation"])
training['Years_from_last_donation'] = where(training["Years_from_last_donation"].isnull(), 0,training["Years_from_last_donation"])

In [795]:
training['is_donation_year_GE_35'] = where(training['Total_Amount_Donated'] > 35, 1,0)
training['is_donation_year_GE_60'] = where(training['Total_Amount_Donated'] > 60, 1,0)
training['is_donation_year_GE_100'] = where(training['Total_Amount_Donated'] > 100, 1,0)

In [796]:
############################################### TEST DATA ###############################################

In [838]:
# Test dataset
campaign2014 = read_csv(inputData + "/campaign20140115.csv", sep = ';')

In [839]:
#Grouping the data by DonorID to find the total amount donated
campaign2014 = campaign2014.groupby(['donorID'],as_index=False).agg('sum')

In [840]:
#Rename Amount Column
campaign2014 = campaign2014.rename(columns= {'amount': 'amount_campaign'})

In [841]:
# gifts dataset
gifts = read_csv(inputData + "/gifts.csv", sep = ';')

In [842]:
#Convert to date format
gifts['date'] = pandas.to_datetime(gifts['date'])

In [843]:
#filtering data before the campaign date
gifts_test = gifts[gifts.date <= '2014-01-15']

In [844]:
#renaming the amount column in Gifts
gifts_test = gifts_test.rename(columns= {'amount': 'amount_gifts'})

In [845]:
#Camp count per year
camp_count_test =gifts_test.groupby(['donorID'], as_index=False)['campID'].count()
camp_count_test=camp_count_test.rename(columns= {'campID': 'Total_camps_attended'})

In [846]:
#column 8
camp_count_test['is_camp_attended'] = where(camp_count_test["Total_camps_attended"] > 0, 1,0)#not required

In [847]:
#column 9
#Total Donation per donor
donation_donor_test =gifts_test.groupby(['donorID'], as_index=False)['amount_gifts'].sum()
donation_donor_test=donation_donor_test.rename(columns= {'amount_gifts': 'Total_Amount_Donated'})

In [848]:
#create a new column for year to obtain the donations per year
gifts_test['gifts_year'] = gifts_test['date'].dt.year

In [849]:
# - to check if it is required qs it has more than one row for donor id
#sum the donations to obtain per year
donation_year_test = gifts_test.groupby(['donorID','gifts_year'],as_index=False).sum()[['amount_gifts']]

In [850]:
#filtering data for the 3 year frequency rate
freq_3_test = gifts_test[(gifts_test['date'] >= '2011-01-15') & (gifts_test['date'] <= '2014-01-15')]
#column 11
#Camp count freq per 3 years
freq_3_camp_test = freq_3_test.groupby(['donorID'], as_index=False)['campID'].count()
freq_3_camp_test['is_freq_3_camp_attended'] = where(freq_3_camp_test["campID"] >0, 1,0)
#column 12
#Donation freq per 3 years
Donation_last_3_years_test = freq_3_test.groupby(['donorID'],as_index=False)['amount_gifts'].sum()
Donation_last_3_years_test=Donation_last_3_years_test.rename(columns= {'amount_gifts': 'Donation_last_3_years'})


In [851]:
#filtering data for the 5 year frequency rate
freq_5_test = gifts_test[(gifts_test['date'] >= '2008-04-11') & (gifts_test['date'] <= '2013-04-11')]
#column 13
##Camp count freq per 5 years
freq_5_camp_test = freq_5_test.groupby(['donorID'],as_index=False)['campID'].count()
freq_5_camp_test['is_freq_5_camp_attended'] = where(freq_5_camp_test["campID"] >0, 1,0)
#column 14
#Donation freq per 5 years
Donation_last_5_years_test = freq_5_test.groupby(['donorID'],as_index=False)['amount_gifts'].sum()
Donation_last_5_years_test=Donation_last_5_years_test.rename(columns= {'amount_gifts': 'Donation_last_5_years'})

In [852]:
#Drop Unwanted Columns
freq_5_camp_test=freq_5_camp_test.drop(['campID'], axis=1)
freq_3_camp_test=freq_3_camp_test.drop(['campID'], axis=1)

In [853]:
#column 15
#max donation by a donor yearwise
max_donation_year_test = gifts_test.groupby(['donorID','gifts_year'],as_index=False).sum()
max_donation_year_test = max_donation_year_test.groupby(['donorID'],as_index=False)['amount_gifts'].max()
max_donation_year_test= max_donation_year_test.rename(columns= {'amount_gifts': 'max_donation_year'})


In [854]:
#min donation by a donor yearwise
min_donation_year_test = gifts_test.groupby(['donorID','gifts_year'],as_index=False).sum()
min_donation_year_test = min_donation_year_test.groupby(['donorID'],as_index=False)['amount_gifts'].min()
min_donation_year_test=min_donation_year_test.rename(columns= {'amount_gifts': 'min_donation_year'})

In [855]:
#column 18
#Max Amount Paid by a donor
Max_Amount_Donated_test=gifts_test.groupby(['donorID'],as_index=False)['amount_gifts'].max()
Max_Amount_Donated_test=Max_Amount_Donated_test.rename(columns= {'amount_gifts': 'Max_Amount_Donated'})
#column 19
#Min Amount Paid by a donor
Min_Amount_Donated_test=gifts_test.groupby(['donorID'],as_index=False)['amount_gifts'].max()
Min_Amount_Donated_test=Min_Amount_Donated_test.rename(columns= {'amount_gifts': 'Min_Amount_Donated'})

In [856]:
#column 21
#Fist Amount Donated
First_Amount_Donated_test=gifts_test.groupby(['donorID'],as_index=False)[['date']].min()
First_Amount_Donated_test=pandas.merge(First_Amount_Donated_test, gifts_test[['donorID', 'date', 'amount_gifts']], left_on=['donorID','date'], right_on = ['donorID','date'])
First_Amount_Donated_test=First_Amount_Donated_test.rename(columns= {'amount_gifts': 'First_Amount_Donated'})
First_Amount_Donated_test=First_Amount_Donated_test.groupby(['donorID','date'],as_index=False)['First_Amount_Donated'].sum()
#column 22
#Last Amount Donated
Last_Amount_Donated_test=gifts_test.groupby(['donorID'],as_index=False)[['date']].max()
Last_Amount_Donated_test=pandas.merge(Last_Amount_Donated_test, gifts_test[['donorID', 'date', 'amount_gifts']], left_on=['donorID','date'], right_on = ['donorID','date'])
Last_Amount_Donated_test=Last_Amount_Donated_test.rename(columns= {'amount_gifts': 'Last_Amount_Donated'})
Last_Amount_Donated_test=Last_Amount_Donated_test.groupby(['donorID','date'],as_index=False)['Last_Amount_Donated'].sum()

In [857]:
#column 23
#Time since first donation
Years_from_first_donation_test = First_Amount_Donated_test
Years_from_first_donation_test['Year']=Years_from_first_donation_test['date'].dt.year
Years_from_first_donation_test['Year']=2013-Years_from_first_donation_test['Year']
Years_from_first_donation_test=Years_from_first_donation_test.rename(columns= {'Year': 'Years_from_first_donation'})

In [858]:
#column 24
#Time since last donation
Years_from_last_donation_test = Last_Amount_Donated_test
Years_from_last_donation_test['Year']=Years_from_last_donation_test['date'].dt.year
Years_from_last_donation_test['Year']=2013-Years_from_last_donation_test['Year']
Years_from_last_donation_test=Years_from_last_donation_test.rename(columns= {'Year': 'Years_from_last_donation'})

In [859]:
#Drop Unwanted Columns
Years_from_last_donation_test=Years_from_last_donation_test.drop(['date'], axis=1)
Years_from_first_donation_test=Years_from_first_donation_test.drop(['date'], axis=1)
Years_from_first_donation_test=Years_from_first_donation_test.drop(['First_Amount_Donated'], axis=1)
Years_from_last_donation_test=Years_from_last_donation_test.drop(['Last_Amount_Donated'], axis=1)

In [860]:
test = campaign2014

In [861]:
testing=pandas.merge(test, donors, on='donorID', how='left')
testing=pandas.merge(testing, camp_count_test, on='donorID', how='left')
testing=pandas.merge(testing, donation_donor_test, on='donorID', how='left')
testing=pandas.merge(testing, freq_3_camp_test, on='donorID', how='left')
testing=pandas.merge(testing, Donation_last_3_years_test, on='donorID', how='left')
testing=pandas.merge(testing, freq_5_camp_test, on='donorID', how='left')
testing=pandas.merge(testing, Donation_last_5_years_test, on='donorID', how='left')
testing=pandas.merge(testing, max_donation_year_test, on='donorID', how='left')
testing=pandas.merge(testing, min_donation_year_test, on='donorID', how='left')
testing=pandas.merge(testing, Max_Amount_Donated_test, on='donorID', how='left')
testing=pandas.merge(testing, Min_Amount_Donated_test, on='donorID', how='left')
testing=pandas.merge(testing, First_Amount_Donated_test, on='donorID', how='left')
testing=pandas.merge(testing, Last_Amount_Donated_test, on='donorID', how='left')
testing=pandas.merge(testing, Years_from_first_donation_test, on='donorID', how='left')
testing=pandas.merge(testing, Years_from_last_donation_test, on='donorID', how='left')


In [862]:
#Drop Unwanted Columns
testing=testing.drop(['gender','language','zipcode','region','province','date_x','Year_x','date_y','Year_y'], axis=1)

In [863]:
#Handle Null Colums in Zipcode
testing['Total_camps_attended'] = where(testing["Total_camps_attended"].isnull(), 0,testing["Total_camps_attended"])
testing['is_camp_attended'] = where(testing["is_camp_attended"].isnull(), 0,testing["is_camp_attended"])
testing['Total_Amount_Donated'] = where(testing["Total_Amount_Donated"].isnull(), 0,testing["Total_Amount_Donated"])
testing['is_freq_3_camp_attended'] = where(testing["is_freq_3_camp_attended"].isnull(), 0,testing["is_freq_3_camp_attended"])
testing['Donation_last_3_years'] = where(testing["Donation_last_3_years"].isnull(), 0,testing["Donation_last_3_years"])
testing['is_freq_5_camp_attended'] = where(testing["is_freq_5_camp_attended"].isnull(), 0,testing["is_freq_5_camp_attended"])
testing['Donation_last_5_years'] = where(testing["Donation_last_5_years"].isnull(), 0,testing["Donation_last_5_years"])
testing['max_donation_year'] = where(testing["max_donation_year"].isnull(), 0,testing["max_donation_year"])
testing['min_donation_year'] = where(testing["min_donation_year"].isnull(), 0,testing["min_donation_year"])
testing['Max_Amount_Donated'] = where(testing["Max_Amount_Donated"].isnull(), 0,testing["Max_Amount_Donated"])
testing['Min_Amount_Donated'] = where(testing["Min_Amount_Donated"].isnull(), 0,testing["Min_Amount_Donated"])
testing['First_Amount_Donated'] = where(testing["First_Amount_Donated"].isnull(), 0,testing["First_Amount_Donated"])
testing['Last_Amount_Donated'] = where(testing["Last_Amount_Donated"].isnull(), 0,testing["Last_Amount_Donated"])
testing['Years_from_first_donation'] = where(testing["Years_from_first_donation"].isnull(), 0,testing["Years_from_first_donation"])
testing['Years_from_last_donation'] = where(testing["Years_from_last_donation"].isnull(), 0,testing["Years_from_last_donation"])

In [864]:
testing['is_donation_year_GE_35'] = where(testing['Total_Amount_Donated'] > 35, 1,0)
testing['is_donation_year_GE_60'] = where(testing['Total_Amount_Donated'] > 60, 1,0)
testing['is_donation_year_GE_100'] = where(testing['Total_Amount_Donated'] > 100, 1,0)

In [865]:
#create Training Set File
training.to_csv(r'C:\Users\sbadhani\Desktop\Predictive Analysis\Project_Data\Data\TrainingSet_DMC.csv')

In [866]:
#create Testing Set File
testing.to_csv(r'C:\Users\sbadhani\Desktop\Predictive Analysis\Project_Data\Data\TestingSet_DMC.csv')